In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score



In [ ]:
from collections import OrderedDict
from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda, SimplifiedIkedaABS
from rolldecayestimators.simplified_ikeda import limits_kawahara

In [ ]:
limits_kawahara

In [ ]:
db = database.get_db()

sql =  """
SELECT * from
loading_conditions
INNER JOIN models
    ON loading_conditions.model_number==models.model_number
    INNER JOIN ships
        ON ships.name == models.ship_name
"""
df_loading_conditions = pd.read_sql_query(sql=sql, con=db.engine)

In [ ]:
loading_condition = df_loading_conditions.median()

In [ ]:
fi_a=np.deg2rad(10)
beam=loading_condition.beam
lpp=loading_condition.lpp
kg=loading_condition.kg
volume=loading_condition.Volume
draught=loading_condition.TA
A0=loading_condition.A0
lBK=loading_condition.BKL
bBK=loading_condition.BKB
scale_factor=loading_condition.scale_factor

In [ ]:
input = pd.Series()
input['g']=9.81
input['omega_hat']=np.mean(limits_kawahara['OMEGA_hat'])
input['fi_a']=fi_a
input['beam']=beam/scale_factor
input['lpp']=lpp/scale_factor
input['kg']=kg/scale_factor
input['volume']=volume/(scale_factor**3)
input['draught']=draught/scale_factor
input['A0']=A0
input['lBK']=lBK/scale_factor
input['bBK']=bBK/scale_factor


In [ ]:
input['w']= lambdas.omega(beam=input['beam'], g=input['g'], omega_hat=input['omega_hat'])
input['Fn']=0.1
input['V'] = input['Fn']*np.sqrt(input['lpp']*input['g'])

In [ ]:
N = 100
data = np.tile(input.values,(N,1))
inputs_raw = pd.DataFrame(data=data, columns=input.index)

In [ ]:
equations.B_1_hat_equation

In [ ]:
def calculate(inputs, IkedaClass=SimplifiedIkeda):
    si = IkedaClass(**inputs)   
    output = inputs.copy()
    output['B_44_hat'] = si.calculate_B44()
    output['B_W0'] =si.calculate_B_W0()
    output['B_W'] =si.calculate_B_W()
    output['B_F'] =si.calculate_B_F()
    output['B_E'] =si.calculate_B_E()
    output['B_BK'] =si.calculate_B_BK()
    output['B_L'] =si.calculate_B_L()
    output['Bw_div_Bw0'] =si.calculate_Bw_div_Bw0()
    
    return output

In [ ]:
Fns=np.linspace(0,0.2,5)
fig,ax=plt.subplots()

for Fn in Fns:

    inputs = inputs_raw.copy()
    inputs['fi_a']=np.deg2rad(np.linspace(1,15,N))
    inputs['V']=Fn*np.sqrt(inputs['lpp']*inputs['g'])
    
    output = calculate(inputs=inputs)
    output['B_44_hat']-=output.iloc[0]['B_44_hat']
    
    output.plot(x='fi_a', y='B_44_hat',label='Fn=%0.2f'%Fn, ax=ax);

In [ ]:
inputs = inputs_raw.copy()
inputs['Fn']=np.linspace(0,0.1,N)
inputs['V']=inputs['Fn']*np.sqrt(inputs['lpp']*inputs['g'])

output = calculate(inputs=inputs)

fig,ax=plt.subplots()
output.plot(x='Fn', y='B_44_hat', ax=ax);

fig,ax=plt.subplots()
output.plot.area(x='Fn', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)

fig,ax=plt.subplots()
output.plot(x='Fn', y='Bw_div_Bw0', ax=ax);
ax.grid(True)

In [ ]:
inputs = inputs_raw.copy()
x='OG/d'
inputs[x]=np.linspace(limits_kawahara[x][0],limits_kawahara[x][1],N)
inputs['OG'] = inputs['OG/d']*inputs['draught']
inputs['kg'] = -inputs['OG'] + inputs['draught'] 

output = calculate(inputs=inputs)

fig,ax=plt.subplots()
output.plot.area(x=x, y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)

In [ ]:
inputs = inputs_raw.copy()
x='fi_a'
inputs[x]=np.deg2rad(np.linspace(1,15,N))

output = calculate(inputs=inputs)

fig,ax=plt.subplots()
output.plot.area(x=x, y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)

In [ ]:
def CB_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df['CB']=values
    df['volume']=df['CB']*df['lpp']*df['beam']*df['draught']
    return df

def BD_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'B/d']=values
    df['beam']=df[r'B/d']*df['draught']
    return df

def OGd_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'OG/d']=values
    df['OG']=df[r'OG/d']*df['draught']
    df['kg']=-df['OG']+df['draught']
    return df

def CMIDchanger(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'A0']=df[r'CMID']=values
    return df

def bBK_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'bBk/B']=values
    df['bBK']=df[r'bBk/B']*df['beam']
    return df

def lBK_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'lBk/LPP']=values
    df['lBK']=df[r'lBk/LPP']*df['lpp']
    return df

def omega_hat_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df['OMEGA_hat']=values
    df['w']=lambdas.omega(beam=input['beam'], g=input['g'], omega_hat=df['OMEGA_hat'])
    return df

def phi_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'fi_a']=np.deg2rad(values)
    return df

def fn_changer(df, limits):
    values = np.linspace(limits[0], limits[1], len(df))
    df[r'Fn']=values
    df['V']=df['Fn']*np.sqrt(df['lpp']*df['g'])
    return df

changers = OrderedDict() 
changers['CB'] = CB_changer
changers[r'B/d'] = BD_changer
changers[r'OG/d'] = OGd_changer
changers['CMID'] = CMIDchanger
changers[r'bBk/B'] = bBK_changer
changers[r'lBk/LPP'] = lBK_changer
changers[r'OMEGA_hat'] = omega_hat_changer
changers[r'fi_a'] = phi_changer
changers[r'Fn'] = fn_changer

labels={
'CB' : r'$C_b$',
r'B/d' : r'$\frac{B}{d}$',
r'OG/d' : r'$\frac{\overline{OG}}{d}$',
'CMID' : r'C_{mid}',
r'bBk/B' : r'$\frac{B_{BK}}{beam}$',
r'lBk/LPP' : r'$\frac{L_{BK}}{L_{pp}}$',
r'OMEGA_hat' : r'\hat{\omega}',
r'fi_a' : r'\phi_a',
r'Fn' : r'F_n',
    
}

limits = dict(limits_kawahara)
limits['fi_a'] = (1,10)
limits['Fn'] = (0,0.2)

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

df_rolldecay_raw = df_rolldecay.copy()

scale_factor=df_rolldecay['scale_factor']
df_rolldecay['lpp']/=scale_factor
df_rolldecay['TA']/=scale_factor 
df_rolldecay['TF']/=scale_factor
df_rolldecay['draught']=(df_rolldecay['TA'] + df_rolldecay['TF'])/2
df_rolldecay['beam']/=scale_factor
df_rolldecay['BKL']/=scale_factor
df_rolldecay['BKB']/=scale_factor
#df_rolldecay['A0']=db_run.loading_condition.A0
df_rolldecay['kg']/=scale_factor
df_rolldecay['Volume']/=(scale_factor**3)
df_rolldecay['gm']/=scale_factor
df_rolldecay['V']=df_rolldecay['ship_speed']*1.852/3.6/np.sqrt(scale_factor)  #[m/s]
df_rolldecay['rho']=1000
df_rolldecay['g']=9.81

df_rolldecay['Fn']=df_rolldecay['V']/np.sqrt(df_rolldecay['lpp']*df_rolldecay['g'])
df_rolldecay['CB']=df_rolldecay['Volume']/(df_rolldecay['lpp']*df_rolldecay['draught']*df_rolldecay['beam'])
df_rolldecay['B/d']=df_rolldecay['beam']/df_rolldecay['draught']
df_rolldecay['OG']=-(df_rolldecay['kg']-df_rolldecay['draught'])
df_rolldecay[r'OG/d']=df_rolldecay['OG']/df_rolldecay['draught']
df_rolldecay['CMID']=df_rolldecay['A0']
df_rolldecay[r'bBk/B']=df_rolldecay['BKB']/df_rolldecay['beam']
df_rolldecay[r'lBk/LPP']=df_rolldecay['BKL']/df_rolldecay['lpp']
df_rolldecay['OMEGA_hat']=lambdas.omega_hat(beam=df_rolldecay.beam, g=df_rolldecay['g'], 
                                            omega0=df_rolldecay['omega0'])


In [ ]:
fig,axes=plt.subplots(ncols=3, nrows=3)
fig.set_size_inches(10,15)

ylims=[]
for ax,(x, changer) in zip(axes.flatten(),changers.items()):
    
    lims=limits[x]
    inputs = inputs_raw.copy()
    inputs = changer(df=inputs, limits=lims)
    
    output = calculate(inputs=inputs, IkedaClass=SimplifiedIkedaABS)
    
    output.plot.area(x=x, y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
    
    # Add model test results:
    if x in df_rolldecay:
        dataframe = df_rolldecay.copy()        
        dataframe['B_e'] = lambdas.B_e_lambda(B_1=dataframe['B_1'], B_2=dataframe['B_2'], 
                                         omega0=dataframe['omega0'], phi_a=inputs.iloc[0]['fi_a'])
        
        dataframe['B_e_hat'] = lambdas.B_hat_lambda(B=dataframe['B_e'], Disp=dataframe['Volume'], 
                                                    beam=dataframe['beam'], 
                                                    g=dataframe['g'], rho=dataframe['rho'])
        dataframe.dropna(subset=['B_e_hat'], inplace=True)
        dataframe.plot(x=x, y='B_e_hat', style='.', ax=ax, label='model test')
    
    if x=='fi_a':
        for phi_a_deg in np.linspace(lims[0], lims[1], 3):        
            dataframe = df_rolldecay.copy()        
            dataframe['fi_a']=np.deg2rad(phi_a_deg)
            dataframe['B_e'] = lambdas.B_e_lambda(B_1=dataframe['B_1'], B_2=dataframe['B_2'], 
                                         omega0=dataframe['omega0'], phi_a= dataframe['fi_a'])
        
            dataframe['B_e_hat'] = lambdas.B_hat_lambda(B=dataframe['B_e'], Disp=dataframe['Volume'], 
                                                    beam=dataframe['beam'], 
                                                    g=dataframe['g'], rho=dataframe['rho'])
        
            dataframe.dropna(subset=['B_e_hat'], inplace=True)
            dataframe.plot(x=x, y='B_e_hat', style='.', ax=ax, label='model test')
    
    xlabel=labels[x]
    ax.set_xlabel(xlabel)
    #ax.grid()
    ax.get_legend().remove()
    ylims.append(ax.get_ylim()[1])    
    
for ax in axes.flatten():
    ax.set_ylim(0,np.max(ylims))
    
plt.tight_layout()
axes[0,2].legend();